# Tutorial 7b: Data Imputation

This covers:

The deletion approach

Deleting the incomplete features
Deleting the incomplete instances
pandas

Simple imputation using pandas
Interpolation imputation using pandas
sklearn

Simple imputation using sklearn
KNN-based imputation using skearn
Iterative imputation using skearn
Applying the learned models to incomplete test data

In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Loading and exploring the data

In [2]:
import pandas as pd
# Or load titanic data that are alraedy split into train and test data sets according to https://www.kaggle.com/c/titanic/data
# But the test data of kaggle does not have labels
# Therefore we will load  the whole data from a data repository then split it latter
titanic_data = pd.read_csv("https://www.openml.org/data/get_csv/16826755/phpMYEkMl.csv", na_values=['?']) #yo
titanic_data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


### Values considered “missing”

There are many ways to represent missing values in both the dataset file and the python pandas.

Missing values in the data might be blank entries, or '?', or something else that data collecters agreed on to represent unobserved data. In this case it is '?' -- knowing this, we tell pandas what to consider as missing values via na_values=['?'].

At the "other end", pandas can represent missing values in several different ways. As can be seen above, "NaN" is the default missing value marker, however, we need to be able to easily detect this value with data of different types: floating point, integer, boolean, and general object. In many cases, however, some other forms can refer to missing values such as None “missing” or “not available”, “NA", or (-)inf .

In [3]:
# Let's drop some features that we will not consider here.
titanic_data.drop(['name','ticket', 'embarked', 'boat' ,'body' ,'home.dest'], axis=1, inplace=True)

Now we will split the data to train and test subsets as ONLY the training data will be used to learn the imputers then the learnt models are applied to the test data

In [4]:
from sklearn.model_selection import train_test_split
y=titanic_data['survived']
X=titanic_data.drop(['survived'], axis=1)
X_titanic_train, X_titanic_test, y_titanic_train, y_titanic_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
#Now if we perform classification it might not work for most classifiers
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
#classifier=SVC()
classifier.fit(X_titanic_train, y_titanic_train)

ValueError: could not convert string to float: 'male'

## There is a problem that some features contain string values, namely the features "sex" and "cabin", so lets encode these features

In [6]:
# We need the upgraded sklearn to accept the parameters for encoders
import sklearn
!pip install -U scikit-learn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 49.1 MB/s  0:00:006m0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.0
    Uninstalling scikit-learn-1.7.0:
      Successfully uninstalled scikit-learn-1.7.0
The scikit-learn version is 1.7.0.


## X_titanic_train_encoded is the encoded incomplete training data

In [8]:
import numpy as np
# Encoding categorical features with preserving the missing values in incomplete features
from sklearn.preprocessing import OrdinalEncoder
encoder_sex = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value=np.nan)
X_titanic_train_encoded=X_titanic_train.copy()
X_titanic_train_encoded['sex'] = encoder_sex.fit_transform(X_titanic_train_encoded['sex'].values.reshape(-1, 1))

#Now lets encode the incomplete Cabin feature
encoder_cabin = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value=np.nan) #You can use the same encoder for both but we use two for the sake of clarfication
X_titanic_train_encoded['cabin'] = encoder_cabin.fit_transform(X_titanic_train_encoded['cabin'].values.reshape(-1, 1).astype(str))
#get the code of the "nan" value for the cabin categorical feature
cabin_nan_code=encoder_cabin.transform([['nan']])[0][0]
#print(cabin_nan_code)
#Now, retrive the nan values to be missing in the encoded data
X_titanic_train_encoded['cabin'].replace(cabin_nan_code,np.nan,inplace=True)

/tmp/ipykernel_1096/3871288400.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_titanic_train_encoded['cabin'].replace(cabin_nan_code,np.nan,inplace=True)


In [9]:
#Check the types of the encoded data, no object features
X_titanic_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 916 entries, 1214 to 1126
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pclass  916 non-null    int64  
 1   sex     916 non-null    float64
 2   age     729 non-null    float64
 3   sibsp   916 non-null    int64  
 4   parch   916 non-null    int64  
 5   fare    915 non-null    float64
 6   cabin   204 non-null    float64
dtypes: float64(4), int64(3)
memory usage: 57.2 KB


In [10]:
# As the data has no strings/object snow, let's try performing classification using the encoded data
classifier.fit(X_titanic_train_encoded, y_titanic_train)

ModuleNotFoundError: No module named 'sklearn.utils._pprint'

ModuleNotFoundError: No module named 'sklearn.utils._pprint'

ModuleNotFoundError: No module named 'sklearn.utils._pprint'

In [12]:
! pip install scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 53.9 MB/s  0:00:006m0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.2
    Uninstalling scikit-learn-1.7.2:
      Successfully uninstalled scikit-learn-1.7.2


In [14]:
! python -m pip show scikit-learn


Name: scikit-learn
Version: 1.5.2
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: BSD 3-Clause License

Copyright (c) 2007-2024 The scikit-learn developers.
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

* Neither the name of the copyright holder nor the names of its
  contributors may be used to endorse or promote products derived from
  this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS 

In [15]:
import sys
print(sys.executable)


/usr/local/python/3.12.1/bin/python3


## Note the error:ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [18]:
print("The number of missing values ")
print(X_titanic_train_encoded.isnull().sum())

The number of missing values 
pclass      0
sex         0
age       187
sibsp       0
parch       0
fare        1
cabin     712
dtype: int64


## The deletion approach

In [19]:
X_titanic_train_complete=X_titanic_train_encoded.copy()
X_titanic_train_complete.dropna(axis=1, inplace=True)
X_titanic_train_complete

,pclass,sex,sibsp,parch
1214,3,1.0,0,0
677,3,1.0,0,0
534,2,0.0,0,0
1174,3,0.0,8,2
864,3,0.0,0,0
...,...,...,...,...
1095,3,0.0,0,0
1130,3,0.0,0,0
1294,3,1.0,0,0
860,3,0.0,0,0


In [20]:
#Check the number of missing values
print(X_titanic_train_complete.isnull().sum())

pclass    0
sex       0
sibsp     0
parch     0
dtype: int64


In [21]:
X_titanic_train_complete=X_titanic_train_encoded.copy()
X_titanic_train_complete.dropna(axis=0, inplace=True)
#The difference is axis=0 instead of 1
X_titanic_train_complete

,pclass,sex,age,sibsp,parch,fare,cabin
39,1,1.0,48.0,0,0,50.4958,14.0
30,1,1.0,45.0,0,0,35.5000,145.0
242,1,0.0,33.0,0,0,27.7208,0.0
136,1,1.0,53.0,0,0,28.5000,68.0
3,1,1.0,30.0,1,2,151.5500,61.0
...,...,...,...,...,...,...,...
189,1,1.0,29.0,0,0,30.0000,113.0
252,1,1.0,61.0,1,3,262.3750,35.0
21,1,0.0,47.0,1,1,52.5542,101.0
276,1,1.0,57.0,1,0,146.5208,42.0


## Notice the reduction in the number of instances
Another important point for the instance deletion approach is that there is a need to remove the target values (from y_train) that correspond to the incomplete (deleted) data instances

In [22]:
#Check the number of missing values
print(X_titanic_train_complete.isnull().sum())

pclass    0
sex       0
age       0
sibsp     0
parch     0
fare      0
cabin     0
dtype: int64



The deletion approach has several drawbacks. It reduces the availlable data, which limits the learning ability, especially when there are many missing values.

Furthermore, the approach of deleting incomplete instances is not practical for test data: we really want to know the answer!

## Imputation using pandas
## Simple imputation (pandas)

In [23]:
X_titanic_data_complete=X_titanic_train_encoded.copy()
X_titanic_data_complete['age']=X_titanic_data_complete['age'].fillna(X_titanic_data_complete['age'].mean())
X_titanic_data_complete['fare']=X_titanic_data_complete['fare'].fillna(X_titanic_data_complete['fare'].mean())
X_titanic_data_complete['cabin']=X_titanic_data_complete['cabin'].fillna(X_titanic_data_complete['cabin'].mean())
# Show the number of missing values
print(X_titanic_data_complete.isnull().sum())

pclass    0
sex       0
age       0
sibsp     0
parch     0
fare      0
cabin     0
dtype: int64


In [24]:
X_titanic_data_complete.head()

,pclass,sex,age,sibsp,parch,fare,cabin
1214,3,1.0,29.102309,0,0,8.6625,73.27451
677,3,1.0,26.000000,0,0,7.8958,73.27451
534,2,0.0,19.000000,0,0,26.0000,73.27451
1174,3,0.0,29.102309,8,2,69.5500,73.27451
864,3,0.0,28.000000,0,0,7.7750,73.27451


## "interpolation" (pandas)

In [26]:
X_titanic_data_complete = X_titanic_train_encoded.copy()
X_titanic_data_complete = X_titanic_data_complete.interpolate()
#The output is 'numpy.ndarray' so we convert it to dataframe for consistency
X_titanic_train_complete = pd.DataFrame(X_titanic_train_complete)
print(X_titanic_train_complete.isna().sum())

pclass    0
sex       0
age       0
sibsp     0
parch     0
fare      0
cabin     0
dtype: int64


##  The default strategy for sklearn simple imputer is the "mean", you can change it using the strategy parameter

## Applying the learned models to incomplete test data

In [33]:

#The learnt encoder_sex should be used to encode the test data, NOTE there is NO fit here, just transform
X_titanic_test_encoded=X_titanic_test.copy()
X_titanic_test_encoded['sex'] = encoder_sex.transform(X_titanic_test_encoded['sex'].values.reshape(-1, 1))

#The learnt encoder2 should be used to encode the test data, NOTE there is NO fit here, just transform
X_titanic_test_encoded['cabin'] = encoder_cabin.transform(X_titanic_test_encoded['cabin'].values.reshape(-1, 1).astype(str))
#Now, retrive the nan values to be missing in the encoded data
X_titanic_test_encoded['cabin'].replace(cabin_nan_code,np.nan,inplace=True)

/tmp/ipykernel_1096/3502827210.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_titanic_test_encoded['cabin'].replace(cabin_nan_code,np.nan,inplace=True)


In [35]:
from sklearn.impute import SimpleImputer

# Khởi tạo imputer, ví dụ thay thế giá trị thiếu bằng giá trị trung bình
imputer = SimpleImputer(strategy='mean')

# Fit imputer trên dữ liệu train
imputer.fit(X_titanic_train_encoded)

# Biến đổi dữ liệu test bằng imputer đã fit
X_titanic_test_complete = imputer.transform(X_titanic_test_encoded)

# Đưa về DataFrame
X_titanic_test_complete = pd.DataFrame(X_titanic_test_complete, 
                                       columns=X_titanic_test_encoded.columns)


ModuleNotFoundError: No module named 'sklearn.impute'

In [34]:
print("The number of missing values in the test data before imputation :\n", X_titanic_test_encoded.isnull().sum())
X_titanic_test_complete = imputer.transform(X_titanic_test_encoded)
X_titanic_test_complete=pd.DataFrame(X_titanic_test_complete, columns=X_titanic_test_encoded.columns)
print("The number of missing values in the test data after imputation :\n", X_titanic_test_complete.isnull().sum())

The number of missing values in the test data before imputation :
 pclass      0
sex         0
age        76
sibsp       0
parch       0
fare        0
cabin     349
dtype: int64


NameError: name 'imputer' is not defined